## Week 4: Handling Complex Images - Happy or Sad Dataset

* 80 150x150 rgb images of emoji-like faces, 40 happy and 40 sad

In [1]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
base_dir = './data/'
happy_dir = os.path.join(base_dir, 'happy/')
sad_dir = os.path.join(base_dir, 'sad/')

In [3]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.999:
            print('\nReached 99.9% accuracy so cancelling training!')
            self.model.stop_training = True

In [4]:
def image_generator():

    train_datagen = ImageDataGenerator(rescale=1/255)

    train_generator = train_datagen.flow_from_directory(directory=base_dir,
                                                        target_size=(150, 150),
                                                        batch_size=10,
                                                        class_mode='binary')

    return train_generator

In [5]:
gen = image_generator()

Found 80 images belonging to 2 classes.


In [6]:
def train_happy_sad_model(train_generator):

    callbacks = myCallback()

    model = tf.keras.Sequential([
        layers.Conv2D(16, 3, activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(1, activation='sigmoid')])

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(train_generator, epochs=15, callbacks=[callbacks])

    return history

In [7]:
history = train_happy_sad_model(gen)

Epoch 1/15
8/8 [==============================] - 2s 24ms/step - loss: 1.6106 - accuracy: 0.5375
Epoch 2/15
8/8 [==============================] - 0s 22ms/step - loss: 0.4639 - accuracy: 0.8000
Epoch 3/15
8/8 [==============================] - 0s 23ms/step - loss: 0.1462 - accuracy: 0.9625
Epoch 4/15
8/8 [==============================] - 0s 23ms/step - loss: 0.0884 - accuracy: 0.9625
Epoch 5/15
8/8 [==============================] - 0s 23ms/step - loss: 0.0489 - accuracy: 0.9875
Epoch 6/15
8/8 [==============================] - 0s 24ms/step - loss: 0.0680 - accuracy: 0.9625
Epoch 7/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0141 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 0s 23ms/step - loss: 0.0127 - accuracy: 1.0000


In [8]:
print(f'The model reached the desired accuracy after {len(history.epoch)} epochs')

The model reached the desired accuracy after 7 epochs


In [9]:
history.model.metrics_names

['loss', 'accuracy']